# Springboard JSON mini project

This notebook will go over the following three tasks:

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,

1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

Import necessary packages, Pandas, JSON, and json_normalize

In [81]:
import pandas as pd
import json
from pandas.io.json import json_normalize

Load the data as a dataframe and take a quick glance

In [82]:
df = pd.read_json(r'C:\Users\Evan\Programming\Jupiter Projects\data_wrangling_json\data\world_bank_projects.json')
df.head(3)

,sector,supplementprojectflg,projectfinancialtype,prodline,mjtheme,idacommamt,impagency,project_name,mjthemecode,closingdate,...,majorsector_percent,board_approval_month,theme_namecode,countryname,url,source,projectstatusdisplay,ibrdcommamt,sector_namecode,_id
0,"[{'Name': 'Primary education'}, {'Name': 'Secondary education'}, {'Name': 'Public administration- Other social services'}, {'Name': 'Tertiary education'}]",N,IDA,PE,[Human development],130000000,MINISTRY OF EDUCATION,Ethiopia General Education Quality Improvement Project II,"8,11",2018-07-07T00:00:00Z,...,"[{'Percent': 46, 'Name': 'Education'}, {'Percent': 26, 'Name': 'Education'}, {'Percent': 16, 'Name': 'Public Administration, Law, and Justice'}, {'Percent': 12, 'Name': 'Education'}]",November,"[{'code': '65', 'name': 'Education for all'}]",Federal Democratic Republic of Ethiopia,http://www.worldbank.org/projects/P129828/ethiopia-general-education-quality-improvement-project-ii?lang=en,IBRD,Active,0,"[{'code': 'EP', 'name': 'Primary education'}, {'code': 'ES', 'name': 'Secondary education'}, {'code': 'BS', 'name': 'Public administration- Other social services'}, {'code': 'ET', 'name': 'Tertiary education'}]",{'$oid': '52b213b38594d8a2be17c780'}
1,"[{'Name': 'Public administration- Other social services'}, {'Name': 'General public administration sector'}]",N,OTHER,RE,"[Economic management, Social protection and risk management]",0,MINISTRY OF FINANCE,TN: DTF Social Protection Reforms Support,"1,6",NaN,...,"[{'Percent': 70, 'Name': 'Public Administration, Law, and Justice'}, {'Percent': 30, 'Name': 'Public Administration, Law, and Justice'}]",November,"[{'code': '24', 'name': 'Other economic management'}, {'code': '54', 'name': 'Social safety nets'}]",Republic of Tunisia,http://www.worldbank.org/projects/P144674?lang=en,IBRD,Active,0,"[{'code': 'BS', 'name': 'Public administration- Other social services'}, {'code': 'BZ', 'name': 'General public administration sector'}]",{'$oid': '52b213b38594d8a2be17c781'}
2,[{'Name': 'Rural and Inter-Urban Roads and Highways'}],Y,IDA,PE,"[Trade and integration, Public sector governance, Environment and natural resources management, Social protection and risk management]",6060000,MINISTRY OF TRANSPORT AND COMMUNICATIONS,Tuvalu Aviation Investment Project - Additional Financing,"5,2,11,6",NaN,...,"[{'Percent': 100, 'Name': 'Transportation'}]",November,"[{'code': '47', 'name': 'Regional integration'}, {'code': '25', 'name': 'Administrative and civil service reform'}, {'code': '81', 'name': 'Climate change'}, {'code': '52', 'name': 'Natural disaster management'}]",Tuvalu,http://www.worldbank.org/projects/P145310?lang=en,IBRD,Active,0,"[{'code': 'TI', 'name': 'Rural and Inter-Urban Roads and Highways'}]",{'$oid': '52b213b38594d8a2be17c782'}


## Task 1: Find top 10 countries with most projects
Manipulate the dataframe to show the shortname of the countries. Each project is listed separately so the count of country names will determine how many projects each country has active.

In [83]:
df.countryshortname.value_counts().nlargest(10)

China                 19
Indonesia             19
Vietnam               17
India                 16
Yemen, Republic of    13
Morocco               12
Nepal                 12
Bangladesh            12
Mozambique            11
Africa                11
Name: countryshortname, dtype: int64

 This lists Africa as a country which needs to be removed.

In [84]:
df_no_continents = df[df.countryshortname != 'Africa']
df_no_continents = df_no_continents.countryshortname.value_counts().nlargest(12)
df_no_continents

Indonesia             19
China                 19
Vietnam               17
India                 16
Yemen, Republic of    13
Morocco               12
Nepal                 12
Bangladesh            12
Mozambique            11
Burkina Faso          9 
Brazil                9 
Pakistan              9 
Name: countryshortname, dtype: int64

The 12 largest are listed because the last three all have 9 active projects.


## Task 2: Find the top 10 major project themes (using column 'mjtheme_namecode')


In [89]:
pd.set_option('display.max_colwidth', -1)
df.mjtheme_namecode.head(10)

0    [{'code': '8', 'name': 'Human development'}, {'code': '11', 'name': ''}]                                                                                                                                                                   
1    [{'code': '1', 'name': 'Economic management'}, {'code': '6', 'name': 'Social protection and risk management'}]                                                                                                                             
2    [{'code': '5', 'name': 'Trade and integration'}, {'code': '2', 'name': 'Public sector governance'}, {'code': '11', 'name': 'Environment and natural resources management'}, {'code': '6', 'name': 'Social protection and risk management'}]
3    [{'code': '7', 'name': 'Social dev/gender/inclusion'}, {'code': '7', 'name': 'Social dev/gender/inclusion'}]                                                                                                                               
4    [{'code': '5', 'name': 'Trade a

Data is stored as dictionaries for codes and names. Some rows have multiple codes/themes and some are even repeated within the same row.
Normalize and view codes, names, and id.

In [90]:
with open(r'C:\Users\Evan\Programming\Jupiter Projects\data_wrangling_json\data\world_bank_projects.json') as file:
    data = json.load(file)
df_mjthemes = json_normalize(data, 'mjtheme_namecode', ['id'])
df_mjthemes.groupby('id').code.nunique().sort_values(ascending=False).head(10)
df_mjthemes.head(5)

,code,name,id
0,8,Human development,P129828
1,11,,P129828
2,1,Economic management,P144674
3,6,Social protection and risk management,P144674
4,5,Trade and integration,P145310


Looks like codes of themes range from 1 - 11 and some names are missing entirely.
Print out top 10 theme names.

In [91]:
df_mjthemes.name.value_counts().head(10)

Environment and natural resources management    223
Rural development                               202
Human development                               197
Public sector governance                        184
Social protection and risk management           158
Financial and private sector development        130
                                                122
Social dev/gender/inclusion                     119
Trade and integration                           72 
Urban development                               47 
Name: name, dtype: int64

## Task 3: Create a dataframe with the missing names filled in.
Fill in missing names based on their codes.


In [92]:
# table for code > name
df_mjthemes_code = df_mjthemes.groupby('name').code.max().sort_values(ascending=False)
# get rid of the missing rows
df_mjthemes_code = df_mjthemes_code[df_mjthemes_code.index != '']
# create dataframe
df_mjthemes_code = pd.DataFrame(df_mjthemes_code,columns=['code'])
# create column for clean names
df_mjthemes_code['name_clean'] = df_mjthemes_code.index
# set code to be the index
df_mjthemes_name = df_mjthemes_code.set_index(['code'])
# merge on the name via code for the missing themes
df_mjthemes.shape
df_mjthemes_cleaned = df_mjthemes.merge(df_themes_code_to_name,how='outer',left_on=['code'],right_index=True)
df_mjthemes_cleaned.shape

(1499, 4)

Top 10 themes with filled in values


In [94]:
df_mjthemes_cleaned.name_clean.value_counts().head(10)

Environment and natural resources management    250
Rural development                               216
Human development                               210
Public sector governance                        199
Social protection and risk management           168
Financial and private sector development        146
Social dev/gender/inclusion                     130
Trade and integration                           77 
Urban development                               50 
Economic management                             38 
Name: name_clean, dtype: int64

Bring cleaned data and original data together

In [96]:
#list of cleaned theme names
df_mjtheme_names = pd.DataFrame(df_mjthemes_cleaned.groupby('id').apply(lambda x: '|'.join(x['name_clean'])),columns=['theme_names'])

df_mjtheme_codes = pd.DataFrame(df_mjthemes.groupby('id').apply(lambda x: '|'.join(x['code'])),columns=['theme_codes'])
#table for merging two dfs
df_mjtheme_table = df_mjtheme_names.merge(df_mjtheme_codes,left_index=True,right_index=True)

df_mjtheme_table.head()

,theme_names,theme_codes
id,,
P075941,Trade and integration|Social dev/gender/inclusion|Financial and private sector development|Rural development,4|7|10|5
P085621,Environment and natural resources management|Environment and natural resources management|Environment and natural resources management|Social dev/gender/inclusion,11|7|11|11
P086592,Environment and natural resources management|Rural development|Rural development|Rural development,11|10|10|10
P094183,Trade and integration|Trade and integration|Rural development,10|5|5
P095003,Trade and integration|Rural development,5|10


Merge lists of themes codes and names for original dataframe

In [100]:
df_orig_clean = df.merge(df_mjtheme_table,how='right',left_on=['id'],right_index=True,)
df_orig_clean[['id', 'mjtheme_namecode', 'theme_names', 'theme_codes']].head()

,id,mjtheme_namecode,theme_names,theme_codes
0,P129828,"[{'code': '8', 'name': 'Human development'}, {'code': '11', 'name': ''}]",Human development|Environment and natural resources management,8|11
1,P144674,"[{'code': '1', 'name': 'Economic management'}, {'code': '6', 'name': 'Social protection and risk management'}]",Economic management|Social protection and risk management,1|6
2,P145310,"[{'code': '5', 'name': 'Trade and integration'}, {'code': '2', 'name': 'Public sector governance'}, {'code': '11', 'name': 'Environment and natural resources management'}, {'code': '6', 'name': 'Social protection and risk management'}]",Environment and natural resources management|Social protection and risk management|Trade and integration|Public sector governance,5|2|11|6
3,P144665,"[{'code': '7', 'name': 'Social dev/gender/inclusion'}, {'code': '7', 'name': 'Social dev/gender/inclusion'}]",Social dev/gender/inclusion|Social dev/gender/inclusion,7|7
4,P144933,"[{'code': '5', 'name': 'Trade and integration'}, {'code': '4', 'name': 'Financial and private sector development'}]",Trade and integration|Financial and private sector development,5|4
